## Finding theoretical quantum parameters of a superconducting cavity-transmon-readout system🥳

We use the program [HFSS by Ansys][HFSS], in addition with the [pyEPR python library][pyEPR] to find the parameters of the system. Without getting too much into details,
the system can be divided into three main parts:

* **The superconducting cavity -** A [Superconducting Radio Frequency (SRF) cavity][SRF], inside which our bosonic qubit is encoded.

* **The Transmon - ** The Transmon is a device based on the [Josephson effect][Josephson], it is used to add non-linearity and qunatumness to the otherwise classical enregy spectrum of the cavity.

* **The read-out resonator - ** The read-out (RO) resonator is simply a strip-line of a superconductor that is coupled to the transmon dispersively, this way we can use
its frequency shifts to measure the state of the transmon.

There are many parameters we may care about in our system, and in this notebook we'll find them with the help of HFSS and pyEPR. The most basic parameters we may care about are the
frequencies of our devices, the cavity and read-out have resonating frequencies based on the *geometry* we set in HFSS while the transmon frequency emarges from the Josephson effect,
and since it is a quantum effect, we need to define the junction parameters in code, and from that get a more precise frequency.

*Note:* Each device can have many resonating frequencies and it might be a bit difficult to distinguish which mode coresponds to which element, there are a couple of ways you can go about it. The easiest is to simply plot the EM field magnitude of each mode and see where the field is the strongest. If the devices are close
to one another they might be difficult to distinguish so another mehod is to use the pyEPR module and find the anharmonicities. Expect the transmon mode to be the only one which is very non-linear (100s of MHz).

Secondly, HFSS also gives the quality factors (equivalent to lifetime using $Q = \omega \tau$) of the modes it finds.

Another set of parameters we may care about are the coupling between the modes, denoted $\chi_{ij}$ for the coupling between the modes $\omega_i$ and $\omega_j$, this is sometimes called
the *cross-kerr* frequencies if $i \ne j$. In the case where $i=j$, $\chi_{ii}$ is the *anharmonicity* of the mode $\omega_i$, usually denoted $\alpha_i$. These parameters are strictly quantum and one has to run pyEPR in order to calculate them.

[HFSS]: https://www.ansys.com/products/electronics/ansys-hfss
[pyEPR]: https://github.com/zlatko-minev/pyEPR
[SRF]: https://en.wikipedia.org/wiki/Superconducting_radio_frequency
[Josephson]: https://en.wikipedia.org/wiki/Josephson_effect

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyEPR as epr

plt.style.use('ggplot')
plt.style.use('seaborn-darkgrid')

#### 🔹 Connecting to HFSS

In [11]:
pinfo = epr.Project_Info(project_path = '.', 
                         project_name = '20210405_CD6',  # File name
                         design_name  = 'CavityCopperPins')

INFO 10:32AM [connect]: Connecting to Ansys Desktop API...
INFO 10:32AM [load_ansys_project]: 	File path to HFSS project found.
INFO 10:32AM [load_ansys_project]: 	Opened Ansys App
INFO 10:32AM [load_ansys_project]: 	Opened Ansys Desktop v2020.2.0
INFO 10:32AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/barkayg/Dropbox (Weizmann Institute)/Quantum Circuits Lab/Experiments/Ket/20210405_CD6/Simulations/
	Project:   20210405_CD6
INFO 10:32AM [connect]: 	Opened active design
	Design:    CavityCopperPins [Solution type: Eigenmode]
INFO 10:32AM [get_setup]: 	Opened setup `Setup1`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 10:32AM [connect]: 	Connection to Ansys established successfully. 😀 



#### 🔹 Run setup (Find modes and Q-factors in HFSS)

In [12]:
%%time
pinfo.setup.analyze()

INFO 10:32AM [analyze]: Analyzing setup Setup1
Wall time: 2min 25s


One can access the simulated results using pyEPR.

In [13]:
eprh = epr.DistributedAnalysis(pinfo)
df = eprh.get_freqs_bare_pd(eprh.variations[0])
df['Lifetime (s)'] = df['Quality Factor']/(2*np.pi*df['Freq. (GHz)']*1e9)
df

Design "CavityCopperPins" info:
	# eigenmodes    5
	# variations    1


,Freq. (GHz),Quality Factor,Lifetime (s)
mode,,,
0,4.367678,4.537884e+05,1.653571e-05
1,5.434226,1.252216e+03,3.667428e-08
2,6.393231,1.738647e+03,4.328239e-08
3,6.492255,6.523918e+07,1.599312e-03
4,6.494370,5.307992e+07,1.300808e-03


Here modes 0, 1 and 2 are the cavity, transmon and RO resonator modes, respectively.

#### 🔹 Define the junction (and any other non-linear component)

In [14]:
pinfo.junctions['j1'] = {'Lj_variable' : 'Lj',
                         'rect'        : 'rect_jj1', 
                         'line'        : 'line_jj1', 
                         'length'      : epr.parse_units('100um')}

# Check that valid names of variables and objects have been supplied.
# An error is raised with a message if something is wrong.
pinfo.validate_junction_info()

#### 🔹 Distributed Analysis (not quantum)

Note that **3 modes is the maximum we can analyze fully❗** See [this issue][issue]

Notice that the modes numbers one wishes to analyze need to be inserted *manually*. They can be identified by looking at the field plots in HFSS.

[issue]: https://github.com/zlatko-minev/pyEPR/issues/36

In [15]:
eprh.do_EPR_analysis(modes=[0,1,2]);


Variation 0  [1/1]

  Mode 0 at 4.37 GHz   [1/5]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
                0.2%  2.804e-21   2.8e-21

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_0j   sign s_0j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 99.20%
	j1              0.00117755  (+)        9.48904e-06
		(U_tot_cap-U_tot_ind)/mean=0.02%

  Mode 1 at 5.43 GHz   [2/5]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
               92.3%  2.293e-24 1.769e-25

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_1j   sign s_1j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 98.77%
	j1              0.919766  (+)        0.0114735
		(U_tot_cap-U_tot_ind)/mean=0.72%

  Mode 2 at 6.39 GHz   [3/5]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
 

#### 🔹 Quantum Analysis (yes quantum)

In [7]:
epra = epr.QuantumAnalysis(eprh.data_filename)
epra.analyze_all_variations(cos_trunc = 8, fock_trunc = 15);

WARNING 10:30AM [__init__]: <p>Error: <class 'IndexError'></p>
	 Differences in variations:



 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Variation 0

Starting the diagonalization
Finished the diagonalization
Pm_norm=
modes
0    1.176632
1    1.015218
2    1.162278
dtype: float64

Pm_norm idx =
      j1
0  False
1   True
2  False
*** P (participation matrix, not normlz.)
         j1
0  0.001178
1  0.909328
2  0.038168

*** S (sign-bit matrix)
   s_j1
0     1
1     1
2     1
*** P (participation matrix, normalized.)
    0.0012
      0.92
     0.038

*** Chi matrix O1 PT (MHz)
    Diag is anharmonicity, off diag is full cross-Kerr.
  0.000108    0.211   0.0103
     0.211      103       10
    0.0103       10    0.244

*** Chi matrix ND (MHz) 
   0.00019    0.319  0.00542
     0.319      111      7.7
   0.00542      7.7    0.155

*** Frequencies O1 PT (MHz)
0    4367.566088
1    5326.145749
2    6387.905960
dtype: float64

*** Frequencies ND (MHz)
0 

### Cross-kerr and anharmonicities

pyEPR just calculated for us all the elemets of $\chi_{ij}$ ($3\times 3$ symmetric matrix)

In [8]:
chis = epra.get_chis()
chis

0           1         2
variation                                  
0         0  0.000190    0.318704  0.005424
          1  0.318704  110.888925  7.703347
          2  0.005424    7.703347  0.154923

The diagonal elements of the matrix are the anharmonicities $\alpha_i$ and the off-diagonal elements are the coupling between the modes.

In [9]:
# anharmonicities
alphas = np.diag(chis)
alpha_c, alpha_t, alpha_r = alphas

# Couplings
chi_ct = chis[1][0]  # cavity-transmon
chi_cr = chis[0][2]  # cavity-readout
chi_tr = chis[1][2]  # transmon-readout

display(pd.DataFrame({'Anharmonicity (MHz)': alphas}))
display(pd.DataFrame({'Coupling (MHz)': [chi_ct, chi_tr, chi_cr]}, index=['cavity-transmon', 'transmon-readout', 'cavity-readout']))

,Anharmonicity (MHz)
0,0.000190
1,110.888925
2,0.154923


,Coupling (MHz)
cavity-transmon,0.318704
transmon-readout,7.703347
cavity-readout,0.005424
